In [96]:
# import libraries
import requests
import time
import datetime
import pandas as pd
import smtplib
from lxml import html
import matplotlib.pyplot as plt
import re
import json

In [ ]:
# connect to website
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.48'}
base_url = 'http://ufcstats.com/statistics/events/completed?page=all'

# base_tree = html.fromstring(requests.get(base_url, headers=headers).content)
response = requests.get(base_url, headers=headers)
print(response)



In [121]:
def get_events(response):
    tree = html.fromstring(response.content)

    events_tbl = tree.xpath('/html/body/section/div/div/div/div[2]/div/table/tbody')[0]
    trs = events_tbl.xpath('.//tr')

    ufc_events = []

    for tr in trs[1:]:
        ufc_event = {
            'name': None,
            'date': None,
            'location': None,
            'link': None
        }
        ufc_event['name'] = tr.xpath('td[1]/i/a/text()')[0].strip()
        ufc_event['date'] = tr.xpath('td[1]/i/span')[0].text.strip()
        ufc_event['link'] = tr.xpath('td[1]/i/a/@href')[0].strip()
        ufc_event['location'] = tr.xpath('td[2]/text()')[0].strip()
        ufc_events.append(ufc_event)

    return ufc_events


In [209]:
events = get_events(response)
ufc_events = pd.DataFrame(events)
ufc_events.to_csv('ufc_events.csv', index=False)


In [290]:
def get_fights(events):
    for event in events[1:2]:
        url = event['link']
        event_name = event['name']
        fight_tree = html.fromstring(requests.get(url, headers=headers).content, 'html.parser')
        fights_tbl = fight_tree.xpath('/html/body/section/div/div/table')[0]
        fight_trs = fights_tbl.xpath('.//tr')

        fights = []
        for tr in fight_trs[1:]:
            fight = {
                'event_name': None,
                'name': None,
                'fighter1': None,
                'fighter2': None,
                'weight class': None,
                'method': None,
                'round': None,
                'time': None,
                'link': None
            }
            
            fight['event_name'] = event_name
            fighter1 = tr.xpath('td[2]/p[1]/a/text()')[0].strip()
            fighter2 = tr.xpath('td[2]/p[2]/a/text()')[0].strip()
            fight['name'] = fighter1 + ' vs ' + fighter2
            fight['fighter1'] = fighter1
            fight['fighter2'] = fighter2
            fight['weight class'] = tr.xpath('td[7]/p/text()')[0].strip()
            fight['method'] = tr.xpath('td[8]/p/text()')[0].strip()
            fight['round'] = tr.xpath('td[9]/p/text()')[0].strip()
            fight['time'] = tr.xpath('td[10]/p/text()')[0].strip()
            fight['link'] = tr.xpath('td[1]/p/a/@href')[0].strip()

            fights.append(fight)

    return fights

In [291]:
fights = get_fights(events)
ufc_fights = pd.DataFrame(fights)
ufc_fights.to_csv('ufc_fights.csv', index=False)

In [ ]:
ufc_fights_records_header = []
for fight in ufc_fights[:4]:
        record_tree = html.fromstring(str(record_soup))
        for record in record_tree.xpath('/html/body/section/div/div'):
                # fighererA = record.xpath('//div[1]/div[1]/div')[0]
                fighterA = BeautifulSoup(html.tostring(record.xpath('//div[1]/div[1]/div/h3/a')[0]), 'html.parser').text.strip()
                fighterB = BeautifulSoup(html.tostring(record.xpath('//div[1]/div[2]/div/h3/a')[0]), 'html.parser').text.strip()
                
                fight_header = 'Fight'
                fight_content = (fighterA + ' vs. ' + fighterB)

                method_header = record.xpath('//div[2]/div[2]/p[1]/i[1]/i[1]')[0].text.strip()
                method_content = record.xpath('//div[2]/div[2]/p[1]/i[1]/i[2]')[0].text.strip()

                round_header = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[2]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[1].text.strip()
                round_content = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[2]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[2].text.strip()

                time_header = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[3]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[1].text.strip()
                time_content = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[3]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[2].text.strip()

                format_header = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[4]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[1].text.strip()
                format_content = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[4]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[2].text.strip()

                referee_header = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[5]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[1].text.strip()
                referee_content = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[5]/span')[0]), 'html.parser').text.strip()

                details_header = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[2]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__label'}).text.strip()
                details_content = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[2]')[0]), 'html.parser').text
                details_content = re.sub(r'\s+', ' ', details_content).split(':')[1].strip()

                individual_fights_record={fight_header: fight_content, method_header: method_content, round_header: round_content, time_header: time_content, format_header: format_content, referee_header: referee_content, details_header: details_content}
                ufc_fights_records_header.append(individual_fights_record)
                # print(fighterA, fighterB)

In [ ]:

# # get all event links
# event_links_element = base_tree.xpath('/html/body/section/div/div/div/div[2]/div/table/tbody//td[1]/i/a')
# event_location_element = base_tree.xpath('/html/body/section/div/div/div/div[2]/div/table/tbody//td[2]')
# event_date_element = base_tree.xpath('/html/body/section/div/div/div/div[2]/div/table/tbody//td[1]/i/span')

# ufc_events = []
# for event_link, location, date in zip(event_links_element, event_location_element, event_date_element):
#     link = event_link.get('href')
#     event = event_link.text.strip()
#     location = location.text.strip()
#     date = date.text.strip()

#     ufc_events.append({'event': event, 'location': location, 'date': date, 'link': link})

# ufc_fights = []
# for ufc_event in ufc_events[:3]:
#         fight_soup = BeautifulSoup(requests.get(ufc_event['link'], headers=headers).content, 'html.parser')
#         fight_tree = html.fromstring(str(fight_soup))
#         ufc_fights_links = fight_tree.xpath('/html/body/section/div/div/table/tbody//td[1]/p/a/@href')
#         for fight_link in ufc_fights_links:
#                 ufc_fights.append({'event': ufc_event['event'], 'fight_link': fight_link})

# print(ufc_fights)

In [ ]:
# # for ufc_fight in ufc_fights:
# ufc_fights_records_header = []
# for ufc_fight in ufc_fights[:4]:
#         record_soup = BeautifulSoup(requests.get(ufc_fight['fight_link'], headers=headers).content, 'html.parser')
#         record_tree = html.fromstring(str(record_soup))
#         for record in record_tree.xpath('/html/body/section/div/div'):
#                 # fighererA = record.xpath('//div[1]/div[1]/div')[0]
#                 fighterA = BeautifulSoup(html.tostring(record.xpath('//div[1]/div[1]/div/h3/a')[0]), 'html.parser').text.strip()
#                 fighterB = BeautifulSoup(html.tostring(record.xpath('//div[1]/div[2]/div/h3/a')[0]), 'html.parser').text.strip()
                
#                 fight_header = 'Fight'
#                 fight_content = (fighterA + ' vs. ' + fighterB)

#                 method_header = record.xpath('//div[2]/div[2]/p[1]/i[1]/i[1]')[0].text.strip()
#                 method_content = record.xpath('//div[2]/div[2]/p[1]/i[1]/i[2]')[0].text.strip()

#                 round_header = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[2]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[1].text.strip()
#                 round_content = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[2]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[2].text.strip()

#                 time_header = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[3]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[1].text.strip()
#                 time_content = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[3]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[2].text.strip()

#                 format_header = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[4]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[1].text.strip()
#                 format_content = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[4]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[2].text.strip()

#                 referee_header = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[5]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__text-item'}).contents[1].text.strip()
#                 referee_content = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[1]/i[5]/span')[0]), 'html.parser').text.strip()

#                 details_header = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[2]')[0]), 'html.parser').find('i', {'class', 'b-fight-details__label'}).text.strip()
#                 details_content = BeautifulSoup(html.tostring(record.xpath('//div[2]/div[2]/p[2]')[0]), 'html.parser').text
#                 details_content = re.sub(r'\s+', ' ', details_content).split(':')[1].strip()

#                 individual_fights_record={fight_header: fight_content, method_header: method_content, round_header: round_content, time_header: time_content, format_header: format_content, referee_header: referee_content, details_header: details_content}
#                 ufc_fights_records_header.append(individual_fights_record)
#                 # print(fighterA, fighterB)

# print(ufc_fights_records_header)

In [ ]:
# ufc_fights_records_header_pd = pd.DataFrame(ufc_fights_records_header)
# ufc_fights_records_header_pd.to_csv('ufc_fights_records_header.csv', index=False)

In [ ]:
# ufc_fights_pd = pd.DataFrame(ufc_fights)
# ufc_fights_pd.to_csv('ufc_fights.csv', index=False)

In [ ]:
# ufc_events_pd = pd.DataFrame(ufc_events)
# ufc_events_pd.to_csv('ufc_events.csv', index=False)